In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/'

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define constants
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS =50
n_classes = 4

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:

# Load the dataset
dataset = datasets.ImageFolder("data", transform=transform)
class_names = dataset.classes
class_names


['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [ ]:
# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
# # Define the CNN model
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.resiz = nn.Sequential(
#             nn.Conv2d(CHANNELS, 32, kernel_size=3, padding=1),  # Adjusted input channels
#             nn.BatchNorm2d(32),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )
#         self.features = nn.Sequential(
#             nn.Conv2d(32, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )
#         self.classifier = nn.Sequential(
#             nn.Linear(64 * 32 * 32, 64),  # Adjusted input size
#             nn.ReLU(inplace=True),
#             nn.Linear(64, n_classes),
#         )

#     def forward(self, x):
#         x = self.resiz(x)
#         x = self.features(x)
#         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         return x

model = models.vgg16(pretrained=True)
# Modify the classifier to fit your classification task
model.classifier[6] = nn.Linear(model.classifier[6].in_features, n_classes)
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Create an instance of the model
#model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters())

# Training loop
train_losses = []
val_losses = []

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_losses.append(val_loss / len(val_loader))

    print(f"Epoch [{epoch + 1}/{EPOCHS}] "
          f"Train Loss: {train_losses[-1]:.4f} "
          f"Validation Loss: {val_losses[-1]:.4f}")

Epoch [1/50] Train Loss: 4.0234 Validation Loss: 1.0038
Epoch [2/50] Train Loss: 0.5582 Validation Loss: 0.4179
Epoch [3/50] Train Loss: 0.4139 Validation Loss: 0.3584
Epoch [4/50] Train Loss: 0.3706 Validation Loss: 0.3687
Epoch [5/50] Train Loss: 0.3680 Validation Loss: 0.3088
Epoch [6/50] Train Loss: 0.3443 Validation Loss: 0.2862
Epoch [7/50] Train Loss: 0.3059 Validation Loss: 0.3017
Epoch [8/50] Train Loss: 0.2864 Validation Loss: 0.2401
Epoch [9/50] Train Loss: 0.2441 Validation Loss: 0.2378
Epoch [10/50] Train Loss: 0.2466 Validation Loss: 0.2813
Epoch [11/50] Train Loss: 0.2342 Validation Loss: 0.2276
Epoch [12/50] Train Loss: 0.2265 Validation Loss: 0.2218
Epoch [13/50] Train Loss: 0.2186 Validation Loss: 0.2167
Epoch [14/50] Train Loss: 0.2096 Validation Loss: 0.2193
Epoch [15/50] Train Loss: 0.2059 Validation Loss: 0.2013
Epoch [16/50] Train Loss: 0.1960 Validation Loss: 0.2541
Epoch [17/50] Train Loss: 0.2358 Validation Loss: 0.2692
Epoch [18/50] Train Loss: 0.1853 Validat

In [ ]:
from sklearn.metrics import classification_report

model.eval()
correct = 0
total = 0
# Calculate and print classification report
true_labels = []
predicted_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:}%")


report = classification_report(true_labels, predicted_labels, target_names=class_names)
print(report)


Accuracy on the test set: 94.52380952380952%
                precision    recall  f1-score   support

        Blight       0.89      0.94      0.91       123
   Common_Rust       1.00      0.93      0.96       129
Gray_Leaf_Spot       0.87      0.87      0.87        55
       Healthy       0.99      1.00      1.00       113

      accuracy                           0.95       420
     macro avg       0.94      0.94      0.94       420
  weighted avg       0.95      0.95      0.95       420

